In [21]:
import numpy as np
from sklearn import svm

from src.utils import load_and_regularize_covariance_matrices, calculate_covariance_means, project_to_tangent_and_triu, evaluate_metrics_repeatedly
from src.covariance_means import generalized_eigenvalue_covariance_mean

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
c_dataset = load_and_regularize_covariance_matrices(deep_features_type='covs_DN20', group_type='CG')
c_dataset_covs = calculate_covariance_means(c_dataset) #Generalized eigenvalue covariance mean
c_dataset_vectors = project_to_tangent_and_triu(c_dataset_covs)

pk_dataset = load_and_regularize_covariance_matrices(deep_features_type='covs_DN20', group_type='PG')
pk_dataset_covs = calculate_covariance_means(pk_dataset)
pk_dataset_vectors = project_to_tangent_and_triu(pk_dataset_covs)

Patients:   0%|          | 0/11 [00:00<?, ?it/s]

Patients [Tangent space]: 100%|██████████| 11/11 [00:00<00:00, 112.18it/s]


In [24]:
# Concatenate control and Pk
dataset_vectors = np.concatenate([c_dataset_vectors, pk_dataset_vectors], axis=0) # (22=parkinsons + 22=control, num_videos, num_features = Tangent space flattened mean covariances)
dataset_labels = np.concatenate([np.zeros(c_dataset_vectors.shape[0]), np.ones(pk_dataset_vectors.shape[0])])


In [27]:
# Evaluate metrics
evaluate_metrics_repeatedly(dataset_vectors = dataset_vectors,
                      dataset_labels = dataset_labels,                      
                      estimator = svm.SVC())

Repetitions:   0%|          | 0/10 [00:00<?, ?it/s]

Repetitions: 100%|██████████| 10/10 [00:00<00:00, 13.53it/s]


,Accuracy,std Accuracy,Precision,std Precision,Recall,std Recall,F1 Score,std F1 Score
0,0.948864,0.183321,0.931818,0.171587,0.90625,0.244797,0.911111,0.232104
